# Assignment 2: Decision Trees

Fill in your name and student ID here.
- Name: Lau Ming Jie
- Student ID: A0272632H

## Overview

In this assignment, we'll implement Decision Trees:

1. **Decision Tree Regression**
    - Compute Region-Residual Sum of Squares (Region-RSS)
    - Decision Tree Regressor
2. **Decision Tree Classification**
    - Calculate Entropy
    - Compute Information Gain
    - Implement the Majority Class function
    - Implement the Best Split function
    - Implement the Recursive Tree Builder Function
    - Implement Prediction Logic for a single data point
    - Decision Tree Classifier
3. **Practical**: Train a DT classifier on the training dataset using scikit-learn

By the end, you'll understand how DT works and how to tackle a problem using DT. Let’s dive in!

## Instructions

1. Fill in your name and student ID at the top of the ipynb file.
2. The parts you need to implement are clearly marked with the following:

    ```
    """ YOUR CODE STARTS HERE """

    """ YOUR CODE ENDS HERE """
    ```

    , and you must **ONLY** write your code in between the above two lines.
3. **IMPORTANT**: Make sure that all of the cells are runnable and can compile without exception, even if the answer is incorrect. This will significantly help us in grading your solutions.
3. For task 1 and 2, you are only allowed to use basic Python functions in your code (no `NumPy` or its equivalents), unless otherwise stated. You may reuse any functions you have defined earlier. If you are unsure whether a particular function is allowed, feel free to ask any of the TAs.
4. For task 3, you may use the `scikit-learn` library.
5. Your solutions will be evaluated against a set of hidden test cases to prevent hardcoding of the answer. You may assume that the test cases are always valid, unless specified otherwise. Partial marks may be given for partially correct solutions.

### Submission Instructions
Items to be submitted:
* **This notebook, NAME-STUID-assignment2.ipynb**: This is where you fill in all your code. Replace "NAME" with your full name and "STUID" with your student ID, which starts with "A", e.g. `"John Doe-A0123456X-assignment2.ipynb"`

Submit your assignment by **Sunday, 14 September 23:59** to Canvas. Points will be deducted late submission.


## Overview



## Task 1 - Decision Tree Regression [4 Points]

### Task 1.1 - Compute Region-Residual Sum of Squares (Region-RSS) [1 Point]

Minimizing $\operatorname{Region-RSS}(l, c)$ helps us find the best split for the decision tree at each step.

$$
\underbrace{\operatorname{Region-RSS}(l,c)}_\text{Assume feature $l$, cutoff $c$}= \underbrace{\operatorname{RSS}(\{X|X_l<c\})}_\text{Left subregion} + \underbrace{\operatorname{RSS}(\{X|X_l\geq c\})}_\text{Right subregion}
$$

In order to do so, we need to calculate the residuals for each sub-region:

$$
\operatorname{RSS}(X) = \sum_{i=1}^{n} \left(y_i - \hat{f}(x_i)\right)^2 = \sum_{i=1}^{n} \left(e_i\right)^2
$$

Implement a function that computes the Region-RSS for a given cutoff, using the target values in the left and right sub-regions (`y_left` and `y_right`), without the use of `numpy`.

**Note**: Make sure that your code is able to handle the case where either `y_left` or `y_right` is an empty list.

In [2]:
# TASK 1.1
def calculate_regionrss(y_left, y_right):
    """
    TODO: Compute the Region-RSS for the split.
    Avoid using NumPy and use only basic Python functions.

    Args:
        y_left: A list of target values in the left split
        y_right: A list of target values in the right split

    Returns:
        Region-RSS error for a given cutoff
    """

    total_rss = 0

    """ YOUR CODE STARTS HERE """
    if not y_left:
        avg_left = 0
    else:
        avg_left = sum(y_left)/len(y_left)

    for i in y_left:
        total_rss += (i - avg_left) ** 2

    if not y_right:
        avg_right = 0
    else:
        avg_right = sum(y_right)/len(y_right)

    for j in y_right:
        total_rss += (j - avg_right) ** 2
    """ YOUR CODE ENDS HERE """

    return total_rss

# TESTCASES 1.1
import math

assert math.isclose(calculate_regionrss([3, 4, 5], [8, 9]), 2.5, rel_tol=1e-5)
assert math.isclose(calculate_regionrss([1, 1, 1], [1, 1]), 0.0, rel_tol=1e-5)
assert math.isclose(calculate_regionrss([], [1, 1]), 0.0, rel_tol=1e-5)
print('All test cases passed!')

All test cases passed!


### Task 1.2 - Decision Tree Regressor [3 Points]

Our Decision Tree Regressor recursively splits the data into two regions at each node, using binary splits that minimize the Region-RSS. Splitting stops when the maximum depth is reached or there is no possible/valid splitting. Each leaf predicts the mean target value of its region.

In [3]:
# TASK 1.2
class DTRegressor:
    def __init__(self, max_depth=2):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        """
        Args:
            X: A list of feature values (one feature per sample).
            y: A list of target values corresponding to each sample.
        """
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        """
        TODO: Implement the recursive tree building logic using RSS.
        Hint: Add to right node when it is exactly the same as the split value.

        Args:
            X: A list of feature values (one feature per sample)
            y: A list of target values corresponding to each sample
            depth: The current depth of the tree

        Returns:
            dict or float:
                If it's an internal node, returns a dictionary representing the node:
                - 'split_value': The feature value at which the split occurs.
                - 'left': The left child node (recursively built tree structure or leaf value).
                - 'right': The right child node (recursively built tree structure or leaf value).
                If it's a leaf node (base case) or max_depth reached, returns the mean of the target values
                in that region, which will be the prediction for that leaf.
        """

        res = None

        """ YOUR CODE STARTS HERE """
        if depth >= self.max_depth or len(set(X)) <= 1 or not X:
            return sum(y)/len(y) if y else 0.0

        xs = sorted(set(X))
        candidates = [(xs[i] + xs[i+1])/2 for i in range(len(xs)-1)]

        best_rss = float("inf")
        best_split = None

        for split_value in candidates:
            y_left = [yt for xt, yt in zip(X, y) if xt < split_value]
            y_right = [yt for xt, yt in zip(X, y) if xt >= split_value]
            rss = calculate_regionrss(y_left, y_right)
            if rss < best_rss:
                best_rss = rss
                best_split = split_value

        X_left, y_left, X_right, y_right = [], [], [], []
        for xt, yt in zip(X, y):
            if xt < best_split:
                X_left.append(xt); y_left.append(yt)
            else:
                X_right.append(xt); y_right.append(yt)

        res = {
            "split_value": best_split,
            "left": self._build_tree(X_left, y_left, depth+1),
            "right": self._build_tree(X_right, y_right, depth+1)
        }
        """ YOUR CODE ENDS HERE """

        return res

    def predict_one(self, x, node=None):
        """
        TODO: Traverse the tree to make a prediction for a single input.
        Hint: Prediction should go to the right child node when it is exactly the same as the split value.

        Args:
            x: A single input feature value.
            node: The current node in the tree (used for recursion)

        Returns:
            A float representing the predicted target value for the input
        """

        prediction = 0

        """ YOUR CODE STARTS HERE """
        if node is None:
            node = self.tree

        if not isinstance(node, dict):
            prediction = float(node)
        else:
            if x < node["split_value"]:
                prediction = self.predict_one(x, node["left"])
            else:
                prediction = self.predict_one(x, node["right"])
        """ YOUR CODE ENDS HERE """

        return prediction

    def predict(self, X):
        """
        TODO: Call predict_one for each input in X and return the predictions.

        Args:
            X: A list of input feature values (one feature per sample)

        Returns:
            A list of predicted target values
        """

        predictions = []

        """ YOUR CODE STARTS HERE """
        for xi in X:
            predictions.append(self.predict_one(xi))
        """ YOUR CODE ENDS HERE """

        return predictions

# TESTCASES 1.2
import math

X = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

model = DTRegressor(max_depth=2)
model.fit(X, y)
predictions = model.predict([1.5, 3.5, 5])
assert all(isinstance(p, float) for p in predictions)
assert len(predictions) == 3
assert math.isclose(predictions[0], 4.0)
assert math.isclose(predictions[1], 9.0)
assert math.isclose(predictions[2], 9.0)


X_offset = [0, 1, 2, 3]
y_offset = [5, 7, 9, 11]
model_offset = DTRegressor(max_depth=2)
model_offset.fit(X_offset, y_offset)
predictions_offset = model_offset.predict([0.5, 2.5])
assert all(isinstance(p, float) for p in predictions_offset)
assert len(predictions_offset) == 2
assert math.isclose(predictions_offset[0], 7.0)
assert math.isclose(predictions_offset[1], 11.0)

print('All test cases passed!')

All test cases passed!


## Task 2 - Decision Tree Classification [11 Points]

### Task 2.1 - Calculate Entropy [1 Point]

The entropy of a label set quantifies the amount of uncertainty or impurity in the distribution of class labels. It is defined as:

$$
H(E) = -\sum_{e\in E}{P\left(e\right)\log_{|E|}\left(P\left(e\right)\right)}
$$

where $P(e)$ is the proportion of samples belonging to class $e$, and $|E|$ is the number of unique classes in the label set. Implement `compute_entropy` using the `math` package.

**Note**: Make sure that your code is able to handle the case where there is only 1 unique class.

In [7]:
# TASK 2.1
import math

def compute_entropy(y, n_unique_classes):
    """
    TODO: Compute class proportions and entropy using the formula.

    Args:
        y: A list of class labels (e.g., ['yes', 'no', 'yes', ...])
        n_unique_classes: The number of unique classes in the dataset

    Returns:
        A float representing the entropy of the label distribution
    """

    entropy = 0

    """ YOUR CODE STARTS HERE """
    total = len(y)
    if total == 0 or n_unique_classes <= 1:
        return 0.0

    counts = {}
    for label in y:
        counts[label] = counts.get(label, 0) + 1

    for count in counts.values():
        p = count / total
        if p > 0:
            entropy -= p * math.log(p, n_unique_classes)
    """ YOUR CODE ENDS HERE """



    return entropy

# TESTCASES 2.1
assert math.isclose(compute_entropy(['yes', 'no', 'yes', 'yes', 'no'], n_unique_classes=2), 0.970950, rel_tol=1e-5)
assert math.isclose(compute_entropy(['yes', 'yes', 'yes','no','maybe','maybe','no','maybe'], n_unique_classes= 3), 0.985056, rel_tol=1e-5)
assert math.isclose(compute_entropy(['cat', 'dog', 'cat', 'fish', 'dog', 'cat'], n_unique_classes= 3), 0.920619, rel_tol=1e-5)
assert math.isclose(compute_entropy(['yes', 'yes', 'yes'], n_unique_classes=1), 0, rel_tol=1e-5)

print('All test cases passed!')

All test cases passed!


### Task 2.2 - Compute Information Gain [1 Point]

The information gain of a split of the attribute $A$ is:

$$
\operatorname{IG}(D,A)=H(D) - \sum_{v\in A} {\frac{|D_v|}{|D|}}H(D_v)=H(D) - \sum_{v\in A} P(D_v)H(D_v)
$$

Implement `information_gain` using the function `compute_entropy` defined before.

**Note**: Make sure that your code is able to handle the case where `parent_y` is an empty list. However, you may assume that each of the list in `list_of_child_ys` is a valid mutually-exclusive split of `parent_y`.

In [11]:
# TASK 2.2
def information_gain(parent_y, list_of_child_ys, n_unique_classes=2):
    """
    TODO: Compute the information gain from the parent to the children

    Args:
        parent_y: Target values of the parent node.
        list_of_child_ys: A list where each element is a list of target values for a child node resulting from a split.
        n_unique_classes: The number of unique classes in the dataset

    Returns:
        A float representing the information gain from the split
    """

    gain = 0

    """ YOUR CODE STARTS HERE """
    if not parent_y:
        return 0

    total_parent = len(parent_y)
    parent_entropy = compute_entropy(parent_y, n_unique_classes)

    weighted_child_entropy = 0
    for child_y in list_of_child_ys:
        weight = len(child_y) / total_parent
        weighted_child_entropy += weight * compute_entropy(child_y, n_unique_classes)

    gain = parent_entropy - weighted_child_entropy
    """ YOUR CODE ENDS HERE """

    return gain

# TESTCASES 2.2
import math

parent = ['yes', 'no', 'yes', 'no']
left = ['yes', 'yes']
right = ['no', 'no']
assert math.isclose(information_gain(parent, [left, right], n_unique_classes = 2), 1.0, rel_tol=1e-5)

parent = ['yes', 'no', 'yes', 'no']
left = ['yes', 'no']
right = ['yes', 'no']
assert math.isclose(information_gain(parent, [left, right], n_unique_classes = 2), 0.0, rel_tol=1e-5)


parent = ['yes', 'no', 'yes', 'no', 'yes', 'no']
child_1 = ['yes', 'yes']
child_2 = ['no', 'no']
child_3 = ['yes', 'no']
assert math.isclose(information_gain(parent, [child_1, child_2, child_3], n_unique_classes=2), 2/3, rel_tol=1e-5)

print('All test cases passed!')

All test cases passed!


### Task 2.3 Implement the Majority Class function [1 Point]

This function finds the most frequent class label within a list of `y_labels`. When a new data point reaches a leaf node in a classification tree, it's assigned the class that's most common among the training examples in that leaf. This function determines that "majority class." Count the occurrences of each label. If there's a tie for the highest count, return the smallest label (alphabetically first) to ensure consistent results.

In [ ]:
# from os import major
# TASK 2.3
def majority_class(y_labels):
        """
        TODO: Implement majority class calculation.

        Args:
            y: A list of class labels

        Returns:
            The majority class label (the one with the highest count), or None if y_labels is empty
            If there is a tie, return the smallest label
        """

        majority_class = None

        """ YOUR CODE STARTS HERE """
        if not y_labels:
            return None

        counts = {}
        for label in y_labels:
            counts[label] = counts.get(label,0) + 1

        max_count = max(counts.values())
        candidates = [label for label, count in counts.items() if count == max_count]

        majority_class = min(candidates)
        """ YOUR CODE ENDS HERE """

        return majority_class

# TESTCASES 2.3
y = ['yes', 'no', 'yes', 'no', 'yes']
assert majority_class(y) == 'yes'

y = ['yes', 'no', 'yes', 'no']
assert majority_class(y) == 'no'

y = [1, 2, 2, 3, 2]
assert majority_class(y) == 2

print('All test cases passed!')

All test cases passed!


### Task 2.4 - Implement the Best Split function [3 Points]

Complete the `find_best_split` function below using the `information_gain` function defined before.

For a given dataset `X` (features) and `y` (labels), it searches through all possible ways to split the data based on each feature and selects the best one A decision tree grows by finding splits that best separate the classes. This function chooses the split with the highest Information Gain (IG), leading to purer child nodes and better classification..

This function must handle both categorical and continuous features:
* Categorical Features
    * String Values like `'red'`, `'blue'`, `'apple'`, etc.
    * Perform a **multi-way split**, creating one child node per unique category (at least 2 categories).
    * DO NOT remove the chosen feature from X after splitting.

* Continuous Features
    * Numeric values like `2.0`, `4.5`, etc.
    * Perform **binary splits** at midpoints between consecutive sorted unique values.

For each potential split:
1. Split the data accordingly.
2. Calculate the information gain from the split.
3. Track the split details (feature index, split type, and child group values).

Return the split with the highest information gain. If there is a tie in information gain, prefer the feature with the lowest index.

As this is quite a complex problem, we suggest that you cross-check your outputs manually and add more test cases to make sure that your code really works!

In [17]:
# TASK 2.4
def find_best_split(X, y, n_unique_classes=2):
    """
    TODO: Implement the best split finding logic for both categorical and continuous features

    Args:
        X: List of feature vectors
        y: List of target labels.
        n_unique_classes: The total number of unique classes in the dataset, used as log base.

    Returns:
        A tuple:
        (
            best_gain,             # float: Highest information gain achieved by any split
            best_feature_idx,      # int: Index of the feature that gives the best split
            best_split_type,       # str: Either 'categorical' or 'continuous'
            best_split_details     # dict: Structure varies based on split type (see below)
        )

        - For a categorical feature, best_split_details should be:
            {
                category_val_1: {'X': [...], 'y': [...]},
                category_val_2: {'X': [...], 'y': [...]},
                ...
            }

        - For a continuous feature, best_split_details should be:
            {
                'split_value': val,        # float: midpoint value used for binary split
                'left_X': [...], 'left_y': [...],
                'right_X': [...], 'right_y': [...]
            }

        If no valid split is found, return:
            (-1.0, None, None, None)
    """

    best_overall_gain = -1.0
    best_overall_feature_index = None
    best_overall_split_type = None
    best_overall_split_details = None

    """ YOUR CODE STARTS HERE """
    if not X or not y:
        return -1, None, None, None

    n_features = len(X[0])
    best_overall_gain = -1

    for feature_idx in range(n_features):
        feature_values = [row[feature_idx] for row in X]
        unique_vals = sorted(set(feature_values))

        if len(unique_vals) <= 1:
            continue

        if isinstance(unique_vals[0], (int, float)):
            candidate_splits = [(unique_vals[i] + unique_vals[i+1]) / 2 for i in range(len(unique_vals)-1)]
            for split_val in candidate_splits:
                left_X, left_y, right_X, right_y = [], [], [], []
                for xi, yi in zip(X, y):
                    if xi[feature_idx] < split_val:
                        left_X.append(xi)
                        left_y.append(yi)
                    else:
                        right_X.append(xi)
                        right_y.append(yi)
                gain = information_gain(y, [left_y, right_y], n_unique_classes)
                if gain > best_overall_gain or (gain == best_overall_gain and feature_idx < best_overall_feature_index):
                    best_overall_gain = gain
                    best_overall_feature_index = feature_idx
                    best_overall_split_type = "continuous"
                    best_overall_split_details = {
                        "split_value": split_val,
                        "left_X": left_X,
                        "left_y": left_y,
                        "right_X": right_X,
                        "right_y": right_y
                    }
        else:
            child_groups = {}
            for val in unique_vals:
                child_X, child_y = [], []
                for xi, yi in zip(X, y):
                    if xi[feature_idx] == val:
                        child_X.append(xi)
                        child_y.append(yi)
                child_groups[val] = {"X": child_X, "y": child_y}
            gain = information_gain(y, [child_groups[val]["y"] for val in unique_vals], n_unique_classes)
            if gain > best_overall_gain or (gain == best_overall_gain and feature_idx < best_overall_feature_index):
                best_overall_gain = gain
                best_overall_feature_index = feature_idx
                best_overall_split_type = "categorical"
                best_overall_split_details = child_groups


    """ YOUR CODE ENDS HERE """

    return best_overall_gain, best_overall_feature_index, best_overall_split_type, best_overall_split_details

# TESTCASES 2.4
import math

# TEST 1: Categorical Feature Split
X = [['red'], ['blue'], ['red'], ['green'], ['blue']]
y = ['yes', 'no', 'yes', 'no', 'no']
n_unique_classes = 2
gain, feature_idx, split_type, split_details = find_best_split(X, y, n_unique_classes)

assert split_type == 'categorical'
assert feature_idx == 0
assert math.isclose(gain, 0.97095, rel_tol=1e-4)
assert isinstance(split_details, dict)
assert set(split_details.keys()) == {'red', 'blue', 'green'}

# TEST 2: Continuous Feature Split
X = [[2.0], [4.0], [6.0], [8.0], [10.0]]
y = ['yes', 'yes', 'no', 'no', 'no']
n_unique_classes = 2
gain, feature_idx, split_type, split_details = find_best_split(X, y, n_unique_classes)

assert split_type == 'continuous'
assert feature_idx == 0
assert isinstance(split_details, dict)
assert 'split_value' in split_details
assert split_details['split_value'] == 5.0  # Midpoint between 4.0 and 6.0
assert math.isclose(gain, 0.97095, rel_tol=1e-4)

# TEST 3: No Valid Split
X = [['same'], ['same'], ['same']]
y = ['yes', 'yes', 'yes']
n_unique_classes = 1
gain, feature_idx, split_type, split_details = find_best_split(X, y, n_unique_classes)

assert gain == -1.0
assert feature_idx is None
assert split_type is None
assert split_details is None

print('All test cases passed!')

All test cases passed!


### Task 2.5 - Implement the Recursive Tree Builder Function [2 Points]

Complete the function `build_tree_recursive` below using the `find_best_split` and `majority_class` functions defined before.

It works recursively to grow the tree branch by branch. At each step, it decides whether to stop growing (i.e., return a leaf node) or to split the data and create internal nodes with children. Use `majority_class` to label leaf nodes and `find_best_split` to decide the best way to divide data at internal nodes.

Base Cases: Stop Recursion When Any of These is True:

- The maximum depth (`max_depth`) is reached.
- There is no further valid split, according to `find_best_split`
- All `y` labels in the current subset are the same class (pure node).

In these cases, return the majority class of the current `y` subset as the leaf node's prediction.

Recursive Step: If No Base Case is Triggered

1. Call `find_best_split(X, y, n_unique_classes)` to determine:
   - Best feature index
   - Type of feature (`'continuous'` or `'categorical'`)
   - Split details (values and data partitions)

2. Construct a dictionary representing an internal node, which includes:
   - `'feature'`: index of the best splitting feature.
   - `'split_type'`: `'continuous'` or `'categorical'`.
   - For continuous features:
     - `'split_value'`: float value for binary split.
     - `'left'` and `'right'`: recursive child nodes.
   - For categorical features:
     - `'children_map'`: a dictionary mapping each category to a recursive child node.
     - `'default_prediction'`: the majority class at this node (used as fallback during inference).

In [18]:
# TASK 2.5
def build_tree_recursive(X, y, depth, max_depth, n_unique_classes):
    """
    TODO: Implement the recursive tree building logic using the best split found.

    Args:
        X: Current subset of feature vectors.
        y: Current subset of target labels.
        depth: Current depth of the tree.
        max_depth: Maximum allowed depth for the tree.
        n_unique_classes: The total number of unique classes in the dataset, used as log base.

    Returns:
        Either:
            - A leaf node: the majority class label (int) if the tree should stop splitting.
            - OR a decision node (dict) with the following structure:

        For continuous features:
            {
                'feature': <feature_index>,                # int, index of feature used for split
                'split_type': 'continuous',               # str
                'split_value': <threshold>,               # float, the midpoint used for binary split
                'left': <left_subtree>,                   # recursive subtree or leaf
                'right': <right_subtree>,                 # recursive subtree or leaf
                'default_prediction': <majority_class>    # int, used for fallback predictions
            }

        For categorical features:
            {
                'feature': <feature_index>,                # int
                'split_type': 'categorical',              # str
                'children_map': {
                    <category_val>: <subtree_or_leaf>,    # one child per category
                    ...
                },
                'default_prediction': <majority_class>    # int
            }
    """

    res = None

    """ YOUR CODE STARTS HERE """
    if depth >= max_depth or not X or len(set(y)) == 1:
        return majority_class(y)

    default_pred = majority_class(y)
    gain, best_feature_index, split_type, split_details = find_best_split(X, y, n_unique_classes)

    if gain <= 0 or best_feature_index is None or split_type is None or split_details is None:
        return default_pred

    if split_type == "continuous":
        left_X = split_details["left_X"]
        left_y = split_details["left_y"]
        right_X = split_details["right_X"]
        right_y = split_details["right_y"]

        res = {
            "feature": best_feature_index,
            "split_type": "continuous",
            "split_value": split_details["split_value"],
            "left": build_tree_recursive(left_X, left_y, depth+1, max_depth, n_unique_classes),
            "right": build_tree_recursive(right_X, right_y, depth+1, max_depth, n_unique_classes),
            "default_prediction": default_pred
        }
    else:
        children_map = {}
        for cat_val, subset in split_details.items():
            children_map[cat_val] = build_tree_recursive(subset["X"], subset["y"], depth+1, max_depth, n_unique_classes)
        res = {
            "feature": best_feature_index,
            "split_type": "categorical",
            "children_map": children_map,
            "default_prediction": default_pred
        }

    """ YOUR CODE ENDS HERE """

    return res

# TESTCASES 2.5

test_cases = [
    # Test Case 1: Base Case - Max depth reached
    {
        "name": "Max depth reached",
        "X": [[1], [2], [3]],
        "y": [0, 1, 0],
        "depth": 2,
        "max_depth": 2,
        "n_unique_classes": 2,
        "expected": 0 # Majority class of [0, 1, 0] is 0
    },
    # Test Case 2: Base Case - All labels are the same
    {
        "name": "All labels same",
        "X": [[1], [2], [3]],
        "y": [0, 0, 0],
        "depth": 0,
        "max_depth": 5,
        "n_unique_classes": 2,
        "expected": 0 # Majority class of [0, 0, 0] is 0
    },
    # Test Case 3: Base Case - Empty X (should return majority of y)
    {
        "name": "Empty X",
        "X": [],
        "y": [0, 1, 0],
        "depth": 0,
        "max_depth": 5,
        "n_unique_classes": 2,
        "expected": 0 # Majority class of [0, 1, 0] is 0
    }
]

# Helper function for running tests
for i, tc in enumerate(test_cases):
    print(f"Running Test Case {i+1}: {tc['name']}")
    result = build_tree_recursive(tc['X'], tc['y'], tc['depth'], tc['max_depth'], tc['n_unique_classes'])

    if "expected_type" in tc:
        if tc["expected_type"] == "continuous_node":
            assert isinstance(result, dict) and result.get('split_type') == 'continuous', \
                f"Test Case {i+1} Failed: Expected continuous node, got {result}"
            print(f"Test Case {i+1} Passed: Correctly identified continuous node.")
        elif tc["expected_type"] == "categorical_node":
            assert isinstance(result, dict) and result.get('split_type') == 'categorical', \
                f"Test Case {i+1} Failed: Expected categorical node, got {result}"
            print(f"Test Case {i+1} Passed: Correctly identified categorical node.")
        elif tc["expected_type"] == "continuous_node_with_leaf_children":
            assert isinstance(result, dict) and result.get('split_type') == 'continuous', \
                f"Test Case {i+1} Failed: Expected continuous node at root, got {result}"
            assert not isinstance(result['left'], dict) and not isinstance(result['right'], dict), \
                f"Test Case {i+1} Failed: Expected leaf children, but found nodes. Left: {result['left']}, Right: {result['right']}"
            print(f"Test Case {i+1} Passed: Correctly built continuous node with leaf children.")
    else:
        assert result == tc['expected'], \
            f"Test Case {i+1} Failed: Expected {tc['expected']}, got {result}"
        print(f"Test Case {i+1} Passed: Correctly returned leaf value {result}.")
    print("-" * 30)

print('All test cases passed!')

Running Test Case 1: Max depth reached
Test Case 1 Passed: Correctly returned leaf value 0.
------------------------------
Running Test Case 2: All labels same
Test Case 2 Passed: Correctly returned leaf value 0.
------------------------------
Running Test Case 3: Empty X
Test Case 3 Passed: Correctly returned leaf value 0.
------------------------------
All test cases passed!


### Task 2.6  - Implement Prediction Logic for a single data point [2 Points]

Complete the `predict_one_instance` function below. It takes a single data point and "walks" it down the decision tree to make a prediction.

Starting from the root node:
- Recursively check the `split_type` (`'continuous'` or `'categorical'`) and the feature used at the current `tree_node`.
- Based on `x_instance`'s value for that feature, decide:
  - whether to go `left` or `right` (for continuous),
  - or which `category` branch to follow (for categorical).

Continue this process until a leaf node, which contains the final predicted class.

Base Case:

- If `tree_node` is not a dictionary, it's a leaf node.
- Return its value directly (the predicted class).

Traversal Logic:

*   Continuous Split:
    - Compare `x_instance[feature_idx]` with `tree_node['split_value']`.
    - If less, recurse into `tree_node['left']`; else recurse into `tree_node['right']`.

*   Categorical Split:
    - Look up `x_instance[feature_idx]` in `tree_node['children_map']`.
    - If found, recurse into the matching child.
    - If unseen category, return `tree_node['default_prediction']`.

In [19]:
# TASK 2.6
def predict_one_instance(x_instance, tree_node):
    """
    TODO: Traverse the decision tree to make a prediction for a single input instance.

    Args:
        x_instance: A single feature vector (e.g., [val1, val2])
        tree_node: The current node of the decision tree (starts with the root)

    Returns:
        The predicted class label
    """

    predicted_label = None

    """ YOUR CODE STARTS HERE """
    if not isinstance(tree_node, dict):
        return tree_node
    feature_idx = tree_node["feature"]

    if tree_node["split_type"] == "continuous":
        if x_instance[feature_idx] < tree_node["split_value"]:
            return predict_one_instance(x_instance, tree_node["left"])
        else:
            return predict_one_instance(x_instance, tree_node["right"])
    else:
        cat_val = x_instance[feature_idx]
        if cat_val in tree_node["children_map"]:
            return predict_one_instance(x_instance, tree_node["children_map"][cat_val])
        else:
            return tree_node["default_prediction"]
    """ YOUR CODE ENDS HERE """

    return predicted_label

# TESTCASES 2.6

# Test Case 1: Build a simple tree with a continuous split and predict.
X_tc1 = [[3.0], [7.0], [2.0], [8.0]]
y_tc1 = [0, 1, 0, 1]
n_unique_classes_tc1 = 2
max_depth_tc1 = 1
tree_tc1 = build_tree_recursive(X_tc1, y_tc1, 0, max_depth_tc1, n_unique_classes_tc1)
# Predict an instance that goes left
prediction_tc1_left = predict_one_instance([2.5], tree_tc1)
assert prediction_tc1_left == 0, f"Test Case 1 Failed (left prediction): Expected 0, got {prediction_tc1_left}"
print("Test Case 1 Passed (build & continuous prediction left)")
# Predict an instance that goes right
prediction_tc1_right = predict_one_instance([6.0], tree_tc1)
assert prediction_tc1_right == 1, f"Test Case 1 Failed (right prediction): Expected 1, got {prediction_tc1_right}"
print("Test Case 1 Passed (build & continuous prediction right)")


# Test Case 2: Build a simple tree with a categorical split and predict.
X_tc2 = [['apple'], ['banana'], ['apple'], ['orange']]
y_tc2 = [0, 1, 0, 1]
n_unique_classes_tc2 = 2
max_depth_tc2 = 1
tree_tc2 = build_tree_recursive(X_tc2, y_tc2, 0, max_depth_tc2, n_unique_classes_tc2)
# Predict a known category ('apple')
prediction_tc2_apple = predict_one_instance(['apple'], tree_tc2)
assert prediction_tc2_apple == 0, f"Test Case 2 Failed (apple prediction): Expected 0, got {prediction_tc2_apple}"
print("Test Case 2 Passed (build & categorical prediction 'apple')")
# Predict an unknown category ('grape') - should use default_prediction
prediction_tc2_grape = predict_one_instance(['grape'], tree_tc2)
assert prediction_tc2_grape == majority_class(y_tc2), f"Test Case 2 Failed (grape prediction): Expected default {majority_class(y_tc2)}, got {prediction_tc2_grape}"
print("Test Case 2 Passed (build & categorical prediction 'grape' - default)")

# Test Case 3: Mixed features - build a tree, and predict a continuous feature value path.
X_tc3 = [[10, 'A'], [2, 'B'], [12, 'A'], [3, 'B']]
y_tc3 = [0, 1, 0, 1]
n_unique_classes_tc3 = 2
max_depth_tc3 = 1
tree_tc3 = build_tree_recursive(X_tc3, y_tc3, 0, max_depth_tc3, n_unique_classes_tc3)
# Predict an instance (continuous feature 0 value: 4, which is <= 6.5)
prediction_tc3 = predict_one_instance([4, 'C'], tree_tc3) # The 'C' doesn't matter for this split
assert prediction_tc3 == 1, f"Test Case 3 Failed: Expected 1, got {prediction_tc3}" # Majority of [1,1] from left branch if split at 6.5
print("Test Case 3 Passed (mixed features - continuous path)")

print('All test cases passed!')

Test Case 1 Passed (build & continuous prediction left)
Test Case 1 Passed (build & continuous prediction right)
Test Case 2 Passed (build & categorical prediction 'apple')
Test Case 2 Passed (build & categorical prediction 'grape' - default)
Test Case 3 Passed (mixed features - continuous path)
All test cases passed!


### Task 2.7 - Decision Tree Classifier [1 Point]

Complete the class below using the `majority_class`, `build_tree_recursive` and `predict_one_instance` functions defined before. We have already imported them for you on Coursemology.

This class brings together all the previous helper functions to create, train, and use a decision tree for classification tasks.

* `fit(self, X, y)`:
    *   Calculate and store the total number of unique classes from `y` in `self.n_unique_classes`.
    *   Otherwise, call `build_tree_recursive` to start building the tree, passing all necessary parameters including `self.max_depth` and `self.n_unique_classes`. Store the resulting tree structure in `self.tree`.
    *   DO NOT return anything!

*   `predict(self, X)`:
    *   If `self.tree` is a leaf node (not a dictionary), simply return a list of that leaf value repeated for each instance in `X`.
    *   Otherwise, iterate through each `x_instance` in the input `X` and call `predict_one_instance` to get a prediction for each. Collect these predictions into a list and return it.

In [21]:
# TASK 2.7
class DTClassifier:
    def __init__(self, max_depth=2):
        """
        Args:
            max_depth (int): The maximum depth of the tree.
        """
        self.max_depth = max_depth
        self.tree = None
        self.n_unique_classes = None # To store the total number of unique classes

    def fit(self, X, y):
        """
        TODO: Fit the decision tree to the training data. DO NOT return anything!

        Args:
            X: A list of feature vectors for training.
            y: A list of corresponding target labels.
        """

        """ YOUR CODE STARTS HERE """
        self.n_unique_classes = len(set(y))

        self.tree = build_tree_recursive(X, y, depth=0, max_depth=self.max_depth, n_unique_classes=self.n_unique_classes)
        """ YOUR CODE ENDS HERE """

    def predict(self, X):
        """
        TODO: Predict class labels for each instance in X using the fitted tree

        Args:
            X: A list of feature vectors for prediction.

        Returns:
            A list of predicted class labels.
        """

        predicted_labels = []

        """ YOUR CODE STARTS HERE """
        if not isinstance(self.tree, dict):
            return [self.tree] * len(X)
        predicted_labels = [predict_one_instance(x_instance, self.tree) for x_instance in X]
        return predicted_labels
        """ YOUR CODE ENDS HERE """

        return predicted_labels

# TESTCASES 2.7

# Test Case 1
X = [[1.0], [2.0], [3.0]]
y = [0, 0, 0]
clf = DTClassifier(max_depth=5)
clf.fit(X, y)
X_predict = [[10.0], [20.0]]
predictions = clf.predict(X_predict)
assert predictions == [0,0] , f"Prediction for a single-leaf tree should return that leaf's value for all instances."

# Test Case 2
X_train = [[3.0], [7.0], [2.0], [8.0]]
y_train = [0, 1, 0, 1]
max_depth = 1
clf = DTClassifier(max_depth=max_depth)
clf.fit(X_train, y_train)
X_predict = [[2.5], [6.0], [4.9], [5.1]]
predictions = clf.predict(X_predict)
assert predictions == [0, 1, 0, 1] , f"Prediction for continuous features should correctly traverse the tree."

# Test Case 3
X_train = [['red'], ['blue'], ['red'], ['green']]
y_train = [0, 1, 0, 1]
max_depth = 1
clf = DTClassifier(max_depth=max_depth)
clf.fit(X_train, y_train)
X_predict = [['red'], ['blue'], ['yellow']]
predictions = clf.predict(X_predict)
assert predictions == [0, 1, majority_class(y_train)] , f"Prediction for categorical features should correctly use children_map and handle unseen categories."

print('All test cases passed!')

All test cases passed!


## Task 3 - Practical [2 Points]

Train a DT classifier on the training dataset using `scikit-learn` and tune its hyperparameters to optimize performance.

You will get full marks if your modelling is appropriate and performs well. But remember, you **MUST NOT** use or access X_test and y_test in your code, as this defeats the purpose of a hidden test set. Any model that does so will be given 0 mark.

Make sure that you have installed `scikit-learn` in your python environment.

**HINT**: Set the `random_state` parameter (if exists) to a certain constant to make your model reproducible (same result on every run)

In [56]:
# TASK 3
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

# Load the iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=41)

def train_model(X_train, y_train):
    """
    TODO: Train and return a DT classifier.

    Args:
        X_train: Training feature vectors
        y_train: Training labels

    Returns:
        A trained sklearn model, your model will be used to predict the labels of test data
    """

    model = None

    """ YOUR CODE STARTS HERE """
    model = DecisionTreeClassifier(max_depth=4,
                                   min_samples_split=3,
                                   min_samples_leaf=3,
                                   random_state=123)

    model.fit(X_train, y_train)
    """ YOUR CODE ENDS HERE """

    return model


# TESTCASES 3
# Our hidden test cases will use your code to train a model to predict the labels of the test data, not necessarily on the same train-test split.
# Note: If your model is poorly designed or performs poorly, points may be deducted.

model = train_model(X_train, y_train)
# Check if the model can predict
predictions = model.predict(X_test)
assert len(predictions) == len(X_test)
accuracy_score = model.score(X_test, y_test)
print(f"Model accuracy: {accuracy_score:.9f}")

Model accuracy: 0.933333333


## END OF ASSIGNMENT